# Universidade Tecnológica Federal do Paraná
## Programa de Pós-Graduação em Computação Aplicada
### Ciência de Dados 2 - 2021/1
### Equipe Evolution:
### Leila Fabiola Ferreira
### Mateus Cichelero da Silva

## Processamentos dos dados de Q&A sobre a pandemia, treinamento do classificador naive bayes e da máquina de buscas

## Descrição:
Considerando a última etapa de EDA do grupo, foram levantadas perguntas da base da Laura referentes a 43 dos principais tópicos encontrados. As perguntas foram rotuladas manualmente e no mínimo 10 variações de cada foram criadas para a base de treinamento supervisionada. 
As respostas apresentadas encontram-se em sites oficiais do Ministério da Saúde (https://www.gov.br/saude/pt-br/coronavirus/perguntas-e-respostas) e Fundação Oswaldo Cruz (https://portal.fiocruz.br/coronavirus/perguntas-e-respostas)

Este notebook apresenta os processamentos realizados até a obtenção dos objetos serializados utilizados na aplicação web que pode ser encontrada em http://covid19-qa-ptbr.herokuapp.com/

In [3]:
#!pip install -U spacy
#!python -m spacy download pt_core_news_sm

In [44]:
import spacy
import string
import nltk
from nltk.corpus import stopwords, wordnet
from nltk.tokenize import word_tokenize

In [5]:
nlp = spacy.load('pt_core_news_md')

In [6]:
import pandas as pd

In [64]:
df_pergunta_topico = pd.read_csv('pergunta_topico.csv')
df_topico_resposta = pd.read_csv('topico_resposta.csv')

In [148]:
df_topico_resposta.head()

,topico,resposta
0,definicao_coronavirus,Os coronavírus são uma grande família de vírus...
1,acoes_doente,"Se estiver doente, com sintomas compatíveis co..."
2,covid_coronavirus,COVID-19 é a doença infecciosa causada pelo no...
3,infeccao,O Brasil vivencia em muitos municípios a trans...
4,grupo_risco,Pessoas idosas e pessoas com condições médicas...


In [65]:
df_pergunta_topico.shape

(447, 2)

In [66]:
df_pergunta_topico.head()

,pergunta,topico
0,O que é o coronavírus?,definicao_coronavirus
1,o que é o novo coronavirus?,definicao_coronavirus
2,o que é o corona?,definicao_coronavirus
3,do que se trata esse vírus?,definicao_coronavirus
4,o que é o vírus?,definicao_coronavirus


In [67]:
df_pergunta_topico['token_lemma'] = df_pergunta_topico['pergunta'].apply(lambda x: [tok.lemma_ for tok in nlp(x)])

In [69]:
df_pergunta_topico.head()

,pergunta,topico,token_lemma
0,O que é o coronavírus?,definicao_coronavirus,"[O, que, ser, o, coronavírus, ?]"
1,o que é o novo coronavirus?,definicao_coronavirus,"[o, que, ser, o, novo, coronavirus, ?]"
2,o que é o corona?,definicao_coronavirus,"[o, que, ser, o, corona, ?]"
3,do que se trata esse vírus?,definicao_coronavirus,"[do, que, se, tratar, esse, vírus, ?]"
4,o que é o vírus?,definicao_coronavirus,"[o, que, ser, o, vírus, ?]"


In [70]:
punc = string.punctuation.replace('?','') 

In [71]:
import unicodedata
import re

"""
A remoção de acentos foi baseada em uma resposta no Stack Overflow.
http://stackoverflow.com/a/517974/3464573
"""

def removerAcentosECaracteresEspeciais(palavra):

    # Unicode normalize transforma um caracter em seu equivalente em latin.
    nfkd = unicodedata.normalize('NFKD', palavra)
    palavraSemAcento = u"".join([c for c in nfkd if not unicodedata.combining(c)])

    # Usa expressão regular para retornar a palavra apenas com números, letras e espaço
    return re.sub('[^a-zA-Z0-9? \\\]', '', palavraSemAcento)

In [86]:
stop_words = set(stopwords.words('portuguese'))
stop_words_en = set(stopwords.words('english'))

In [73]:
df_pergunta_topico['lower'] = df_pergunta_topico['token_lemma'].apply(lambda x: [word.lower() for word in x])

In [74]:
df_pergunta_topico['no_punc'] = df_pergunta_topico['lower'].apply(lambda x: [word for word in x if word not in punc])
df_pergunta_topico['no_punc_accen'] = df_pergunta_topico['no_punc'].apply(lambda x: [removerAcentosECaracteresEspeciais(word) for word in x])
df_pergunta_topico['stopwords_removed'] = df_pergunta_topico['no_punc_accen'].apply(lambda x: [word for word in x if word not in stop_words])

In [75]:
df_pergunta_topico.head()

,pergunta,topico,token_lemma,lower,no_punc,no_punc_accen,stopwords_removed
0,O que é o coronavírus?,definicao_coronavirus,"[O, que, ser, o, coronavírus, ?]","[o, que, ser, o, coronavírus, ?]","[o, que, ser, o, coronavírus, ?]","[o, que, ser, o, coronavirus, ?]","[ser, coronavirus, ?]"
1,o que é o novo coronavirus?,definicao_coronavirus,"[o, que, ser, o, novo, coronavirus, ?]","[o, que, ser, o, novo, coronavirus, ?]","[o, que, ser, o, novo, coronavirus, ?]","[o, que, ser, o, novo, coronavirus, ?]","[ser, novo, coronavirus, ?]"
2,o que é o corona?,definicao_coronavirus,"[o, que, ser, o, corona, ?]","[o, que, ser, o, corona, ?]","[o, que, ser, o, corona, ?]","[o, que, ser, o, corona, ?]","[ser, corona, ?]"
3,do que se trata esse vírus?,definicao_coronavirus,"[do, que, se, tratar, esse, vírus, ?]","[do, que, se, tratar, esse, vírus, ?]","[do, que, se, tratar, esse, vírus, ?]","[do, que, se, tratar, esse, virus, ?]","[tratar, virus, ?]"
4,o que é o vírus?,definicao_coronavirus,"[o, que, ser, o, vírus, ?]","[o, que, ser, o, vírus, ?]","[o, que, ser, o, vírus, ?]","[o, que, ser, o, virus, ?]","[ser, virus, ?]"


In [77]:
df_pergunta_topico['stopwords_removed'].min()

['acoes', 'dever', 'tomar', 'estar', 'doente', '?']

# Preparando tradução das perguntas-base para inglês:

In [78]:
from translate import Translator

In [79]:
translator= Translator(from_lang='pt',to_lang="en")

In [82]:
df_pergunta_topico['pergunta_en'] = df_pergunta_topico['pergunta'].apply(lambda x: translator.translate(x))

In [83]:
df_pergunta_topico.head()

,pergunta,topico,token_lemma,lower,no_punc,no_punc_accen,stopwords_removed,pergunta_en
0,O que é o coronavírus?,definicao_coronavirus,"[O, que, ser, o, coronavírus, ?]","[o, que, ser, o, coronavírus, ?]","[o, que, ser, o, coronavírus, ?]","[o, que, ser, o, coronavirus, ?]","[ser, coronavirus, ?]",What is coronavirus?
1,o que é o novo coronavirus?,definicao_coronavirus,"[o, que, ser, o, novo, coronavirus, ?]","[o, que, ser, o, novo, coronavirus, ?]","[o, que, ser, o, novo, coronavirus, ?]","[o, que, ser, o, novo, coronavirus, ?]","[ser, novo, coronavirus, ?]",what is the new coronavirus?
2,o que é o corona?,definicao_coronavirus,"[o, que, ser, o, corona, ?]","[o, que, ser, o, corona, ?]","[o, que, ser, o, corona, ?]","[o, que, ser, o, corona, ?]","[ser, corona, ?]",what is the corona?
3,do que se trata esse vírus?,definicao_coronavirus,"[do, que, se, tratar, esse, vírus, ?]","[do, que, se, tratar, esse, vírus, ?]","[do, que, se, tratar, esse, vírus, ?]","[do, que, se, tratar, esse, virus, ?]","[tratar, virus, ?]",what is this virus about?
4,o que é o vírus?,definicao_coronavirus,"[o, que, ser, o, vírus, ?]","[o, que, ser, o, vírus, ?]","[o, que, ser, o, vírus, ?]","[o, que, ser, o, virus, ?]","[ser, virus, ?]",what is the virus?


In [84]:
df_pergunta_topico['pergunta_en'].max()

'what should i do if i am sick?'

In [85]:
nlp_en = spacy.load('en_core_web_sm')

In [87]:
df_pergunta_topico['token_lemma_en'] = df_pergunta_topico['pergunta_en'].apply(lambda x: [tok.lemma_ for tok in nlp_en(x)])
df_pergunta_topico['lower_en'] = df_pergunta_topico['token_lemma_en'].apply(lambda x: [word.lower() for word in x])
df_pergunta_topico['stopwords_removed_en'] = df_pergunta_topico['lower_en'].apply(lambda x: [word for word in x if word not in stop_words_en])

In [88]:
df_pergunta_topico.head()

,pergunta,topico,token_lemma,lower,no_punc,no_punc_accen,stopwords_removed,pergunta_en,token_lemma_en,lower_en,stopwords_removed_en
0,O que é o coronavírus?,definicao_coronavirus,"[O, que, ser, o, coronavírus, ?]","[o, que, ser, o, coronavírus, ?]","[o, que, ser, o, coronavírus, ?]","[o, que, ser, o, coronavirus, ?]","[ser, coronavirus, ?]",What is coronavirus?,"[what, be, coronavirus, ?]","[what, be, coronavirus, ?]","[coronavirus, ?]"
1,o que é o novo coronavirus?,definicao_coronavirus,"[o, que, ser, o, novo, coronavirus, ?]","[o, que, ser, o, novo, coronavirus, ?]","[o, que, ser, o, novo, coronavirus, ?]","[o, que, ser, o, novo, coronavirus, ?]","[ser, novo, coronavirus, ?]",what is the new coronavirus?,"[what, be, the, new, coronavirus, ?]","[what, be, the, new, coronavirus, ?]","[new, coronavirus, ?]"
2,o que é o corona?,definicao_coronavirus,"[o, que, ser, o, corona, ?]","[o, que, ser, o, corona, ?]","[o, que, ser, o, corona, ?]","[o, que, ser, o, corona, ?]","[ser, corona, ?]",what is the corona?,"[what, be, the, corona, ?]","[what, be, the, corona, ?]","[corona, ?]"
3,do que se trata esse vírus?,definicao_coronavirus,"[do, que, se, tratar, esse, vírus, ?]","[do, que, se, tratar, esse, vírus, ?]","[do, que, se, tratar, esse, vírus, ?]","[do, que, se, tratar, esse, virus, ?]","[tratar, virus, ?]",what is this virus about?,"[what, be, this, virus, about, ?]","[what, be, this, virus, about, ?]","[virus, ?]"
4,o que é o vírus?,definicao_coronavirus,"[o, que, ser, o, vírus, ?]","[o, que, ser, o, vírus, ?]","[o, que, ser, o, vírus, ?]","[o, que, ser, o, virus, ?]","[ser, virus, ?]",what is the virus?,"[what, be, the, virus, ?]","[what, be, the, virus, ?]","[virus, ?]"


In [90]:
df_pergunta_topico['stopwords_removed_en'].min()

['action', 'take', 'infect', '?']

In [91]:
df_pergunta_topico.to_csv('pergunta_topico_processado.csv', header=True, index=False)

# Treinando um classificador para perguntas - respostas

In [92]:
df_classificador = df_pergunta_topico[['stopwords_removed', 'topico']]

In [93]:
df_classificador.head()

,stopwords_removed,topico
0,"[ser, coronavirus, ?]",definicao_coronavirus
1,"[ser, novo, coronavirus, ?]",definicao_coronavirus
2,"[ser, corona, ?]",definicao_coronavirus
3,"[tratar, virus, ?]",definicao_coronavirus
4,"[ser, virus, ?]",definicao_coronavirus


In [94]:
lista_de_topicos = list(df_classificador['topico'].unique())

In [102]:
len(lista_de_topicos)

43

In [105]:
lista_de_topicos[0]

'definicao_coronavirus'

In [101]:
from sklearn.model_selection import train_test_split

In [124]:
X = df_classificador[['stopwords_removed', 'topico']]
y = df_classificador.topico

In [125]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state = 42,stratify=y)

/home/mateuscichelero/miniconda3/envs/ds/lib/python3.7/site-packages/sklearn/model_selection/_split.py:1609: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  return floored.astype(np.int)
/home/mateuscichelero/miniconda3/envs/ds/lib/python3.7/site-packages/sklearn/model_selection/_split.py:1609: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify

In [126]:
X_train.head()

,stopwords_removed,topico
338,"[imunidade, duro, quantum, tempo, tomar, vacin...",tempo_imunidade
111,"[ir, ter, tomar, vacinar, quantum, quantum, te...",vacina_periodicidade
304,"[onde, poder, comprar, vacinar, ?]",comprar_vacina
362,"[ser, casar, reinfeccao, ?]",reinfeccao
231,"[dever, esperar, algum, tempo, tomar, vacinar,...",outras_vacinas_juntas


In [113]:
y_train.shape

(335,)

In [114]:
y_test.shape

(112,)

In [244]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
import pickle

In [133]:
def dummy(doc):
    return doc

In [134]:
nb = Pipeline([('vect', CountVectorizer(tokenizer=dummy,preprocessor=dummy)),
               ('tfidf', TfidfTransformer()),
               ('clf', MultinomialNB()),
              ])
nb.fit(X_train['stopwords_removed'], y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1),
        preprocessor=<function dummy at 0x7fc8b9d...inear_tf=False, use_idf=True)), ('clf', MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))])

In [245]:
# save the model to disk
filename = 'nb_pipeline.pkl'
pickle.dump(nb, open(filename, 'wb'))

In [136]:

%%time
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score, confusion_matrix

y_pred = nb.predict(X_test['stopwords_removed'])

print('accuracy %s' % accuracy_score(y_pred, y_test))
print(classification_report(y_test, y_pred,target_names=lista_de_topicos))

accuracy 0.8482142857142857
                          precision    recall  f1-score   support

   definicao_coronavirus       0.60      1.00      0.75         3
            acoes_doente       1.00      1.00      1.00         3
       covid_coronavirus       1.00      1.00      1.00         2
                infeccao       0.60      1.00      0.75         3
             grupo_risco       0.60      1.00      0.75         3
        virus_superficie       1.00      1.00      1.00         3
             uso_mascara       1.00      1.00      1.00         3
        seguranca_vacina       1.00      1.00      1.00         3
        imunidade_vacina       1.00      1.00      1.00         2
   vacina_mutacoes_virus       0.00      0.00      0.00         1
   vacina_efeito_adverso       1.00      1.00      1.00         3
    vacina_periodicidade       1.00      1.00      1.00         3
  vacina_tempo_imunidade       0.00      0.00      0.00         2
    vacinado_transmissao       1.00      1.00  

/home/mateuscichelero/miniconda3/envs/ds/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [246]:
# save the language_model to disk
filename = 'nlp_model.pkl'
pickle.dump(nlp, open(filename, 'wb'))

In [162]:
def tokeniza_lemmatiza(pergunta):
     return [tok.lemma_ for tok in nlp(pergunta)]
    
def to_lower(token_lemma):
    return [word.lower() for word in token_lemma]

def no_punc(lowered):
    return [word for word in lowered if word not in punc]

def no_accen(no_punc):
    return [removerAcentosECaracteresEspeciais(word) for word in no_punc]

def stopword_removed(no_accen):
    return [word for word in no_accen if word not in stop_words]

In [209]:
pergunta = 'máscara feita em casa protege do vírus?'

In [ ]:
# traduz para inglês

In [210]:
entrada = stopword_removed(
    no_accen(
        no_punc(
            to_lower(
                tokeniza_lemmatiza(pergunta)))))

In [211]:
topico= nb.predict([entrada])[0]
print(f'A pergunta foi classificada como sendo do tópico: {topico}')
print()
print()
print('Resposta indicada de uma fonte oficial sobre o tema da pergunta: ')
print()
print(df_topico_resposta['resposta'][df_topico_resposta['topico']==topico].iloc[0])

A pergunta foi classificada como sendo do tópico: uso_mascara


Resposta indicada de uma fonte oficial sobre o tema da pergunta: 

Dados científicos recentes constatam que a transmissão da COVID-19 pode ocorrer mesmo antes do indivíduo apresentar os primeiros sinais e sintomas. Por esse motivo, o Ministério da Saúde passou a recomendar o uso de máscaras faciais para todos. A utilização de máscaras impede a disseminação de gotículas expelidas do nariz ou da boca do usuário no ambiente, garantindo uma barreira física que vem auxiliando na mudança de comportamento da população e diminuição de casos. 

Diante da insuficiência de insumos, foi solicitado aos cidadãos para que produzam a sua própria máscara de tecido, deixando as máscaras profissionais (cirúrgica e N95 ou similares) para os profissionais da saúde ou outro grupo de trabalhador para o qual exista a previsão legal do uso deste EPI. 

O uso da máscara caseira ajuda na prevenção da COVID-19 desde que associada a outras medidas de 

# Aplicando chatbot - recuperação de informação via similaridade de cossenos

In [212]:
import io
import random
import string # to process standard python strings
import warnings
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import warnings
warnings.filterwarnings('ignore')

In [234]:
f=open('respostas_ministerio.txt','r',errors = 'ignore')
raw=f.read()
raw = raw.lower()# converts to lowercase

In [235]:
import nltk

In [236]:
sent_tokens = nltk.sent_tokenize(raw)# converts to list of sentences 
word_tokens = nltk.word_tokenize(raw)# converts to list of words

In [237]:
lemmer = nltk.stem.WordNetLemmatizer()
#WordNet is a semantically-oriented dictionary of English included in NLTK.
def LemTokens(tokens):
    return [lemmer.lemmatize(token) for token in tokens]
remove_punct_dict = dict((ord(punct), None) for punct in string.punctuation)

def LemNormalize(text):
    return LemTokens(nltk.word_tokenize(text.lower().translate(remove_punct_dict)))

In [238]:
SAUDACOES_INPUTS = ("oi", "olá", "ola", "bom dia", "boa tarde","hey","boa noite", "saudações", "saudacoes", "oi, tudo bem?")
SAUDACOES_RESPOSTAS = ["oi", "ola", "ola voce", "tudo bom?", "feliz em falar contigo!"]
def greeting(sentence):
    for word in sentence.split():
        if word.lower() in SAUDACOES_INPUTS:
            return random.choice(SAUDACOES_RESPOSTAS)

In [239]:
def response(user_response):
    robo_response=''
    sent_tokens.append(user_response)
    TfidfVec = TfidfVectorizer(tokenizer=LemNormalize, stop_words=stop_words)
    tfidf = TfidfVec.fit_transform(sent_tokens)
    vals = cosine_similarity(tfidf[-1], tfidf)
    idx=vals.argsort()[0][-2]
    flat = vals.flatten()
    flat.sort()
    req_tfidf = flat[-2]
    if(req_tfidf==0):
        robo_response=robo_response+"Desculpe! Nao te entendi"
        return robo_response
    else:
        robo_response = robo_response+sent_tokens[idx]
        return robo_response

In [240]:
import os
clear = lambda: os.system('clear')

In [241]:
flag=True
print("ROBIN: Meu nome é Robin. Hoje estarei respondendo suas perguntas sobre o coronavirus. Se quiser finalizar a conversa, basta escrever Tchau!")
while(flag==True):
    user_response = input()
    clear()
    user_response=user_response.lower()
    if(user_response!='tchau'):
        if(user_response=='obrigado' or user_response=='valeu' or user_response=='muito obrigado'):
            flag=False
            print("ROBIN: De nada..")
        else:
            if(greeting(user_response)!=None):
                print("ROBIN: "+greeting(user_response))
            else:
                print("ROBIN: ",end="")
                print(response(user_response))
                sent_tokens.remove(user_response)
    else:
        flag=False
        print("ROBIN: Até logo! Fique bem e se cuide..")

ROBIN: Meu nome é Robin. Hoje estarei respondendo suas perguntas sobre o coronavirus. Se quiser finalizar a conversa, basta escrever Tchau!


 oi


ROBIN: feliz em falar contigo!


 sintomas


ROBIN: isso pode acontecer mesmo sem o desenvolvimento de sintomas.


...

ROBIN: isso pode acontecer mesmo sem o desenvolvimento de sintomas.


 animais transmitem a doença?


ROBIN: estudos em animais não demonstraram risco.


 tchau


ROBIN: Até logo! Fique bem e se cuide..
